In [70]:
import os
import pandas as pd
from datetime import datetime
org_path = os.getcwd()
data_path = org_path + '/stock-data'
data_csv = [csv for csv in os.listdir(data_path) if csv.endswith('.csv')]
print (data_csv)

['0005_trade_20161001-20170112.csv', '0494_trade_20161001-20170112.csv', '0700_trade_20161001-20170112.csv', '0939_trade_20161001-20170112.csv', '0941_trade_20161001-20170112.csv', '1299_trade_20161001-20170112.csv', '2318_trade_20161001-20170112.csv', '2333_trade_20161001-20170112.csv', '2628_trade_20161001-20170112.csv']


In [98]:
dt_0005 = pd.read_csv('%s/0005_trade_20161001-20170112.csv' % (data_path))
dt_0005['Date'] = pd.to_datetime(dt_0005['UpdatedTime'], format="%Y-%m-%d").dt.date
dt_0005['Time'] = pd.to_datetime(dt_0005['UpdatedTime'],format= '%Y-%m-%d %H:%M:%S.%f').dt.time
cols = dt_0005.columns.tolist()
cols = cols[-2:] + cols[:-2]
dt_0005 = dt_0005[cols]
del dt_0005['UpdatedTime']
del dt_0005['Side']
print (dt_0005.head())
print (dt_0005.shape)

         Date      Time  Symbol  Price  Quantity
0  2016-10-03  09:20:00       5  58.15      2400
1  2016-10-03  09:20:00       5  58.15      5200
2  2016-10-03  09:20:00       5  58.15      1200
3  2016-10-03  09:20:00       5  58.15       400
4  2016-10-03  09:20:00       5  58.15       800
(549785, 5)


In [99]:
opening = datetime.strptime('09:30:00', '%H:%M:%S').time()
dt_0005 = dt_0005.loc[dt_0005['Time'] >= opening]
closing = datetime.strptime('16:00:00', '%H:%M:%S').time()
dt_0005 = dt_0005.loc[dt_0005['Time'] <= closing]

In [100]:
print (dt_0005.head())
print (dt_0005.shape)

           Date      Time  Symbol  Price  Quantity
133  2016-10-03  09:30:00       5  58.15     56400
134  2016-10-03  09:30:00       5  58.10       800
135  2016-10-03  09:30:00       5  58.10      1200
136  2016-10-03  09:30:00       5  58.10       400
137  2016-10-03  09:30:00       5  58.15     20000
(533612, 5)
